In [1]:
conda install -c anaconda graphviz

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
uri = 'https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv'

data = pd.read_csv(uri, index_col = 0)
data.head()

,mileage_per_year,model_year,price,sold
0,21801,2000,30941.02,yes
1,7843,1998,40557.96,yes
2,7109,2006,89627.50,no
3,26823,2015,95276.14,no
4,7935,2014,117384.68,yes


In [3]:
novo_nome = {'mileage_per_year':'milhas_por_ano', 'model_year':'ano_do_modelo', 'price':'preco', 'sold':'vendido'}
trocar = {'yes':1, 'no':0}

data = data.rename(columns = novo_nome)
data.vendido = data.vendido.map(trocar)
data.head()

,milhas_por_ano,ano_do_modelo,preco,vendido
0,21801,2000,30941.02,1
1,7843,1998,40557.96,1
2,7109,2006,89627.50,0
3,26823,2015,95276.14,0
4,7935,2014,117384.68,1


In [4]:
from datetime import datetime as dt
ano_atual = dt.today().year

data['idade_do_modelo'] = ano_atual - data.ano_do_modelo
data.head()

,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,21801,2000,30941.02,1,19
1,7843,1998,40557.96,1,21
2,7109,2006,89627.50,0,13
3,26823,2015,95276.14,0,4
4,7935,2014,117384.68,1,5


In [5]:
data['km_por_ano'] = data['milhas_por_ano'] * 1.60934
data.head()

,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,21801,2000,30941.02,1,19,35085.22134
1,7843,1998,40557.96,1,21,12622.05362
2,7109,2006,89627.50,0,13,11440.79806
3,26823,2015,95276.14,0,4,43167.32682
4,7935,2014,117384.68,1,5,12770.11290


In [6]:
data = data.drop(columns = ['milhas_por_ano', 'ano_do_modelo'], axis = 1)
data.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,19,35085.22134
1,40557.96,1,21,12622.05362
2,89627.50,0,13,11440.79806
3,95276.14,0,4,43167.32682
4,117384.68,1,5,12770.11290


In [7]:
x = data[['preco', 'idade_do_modelo', 'km_por_ano']]
y = data['vendido']

from sklearn.svm import LinearSVC as svc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as tt_split
import numpy as np

np.random.seed(5)
treino_x, teste_x, treino_y, teste_y = tt_split(x, y, test_size = .25, stratify = y)
print(f'Treino = {treino_x.shape[0]} e Teste = {teste_x.shape[0]}')

modelo = svc(dual = False)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) *100
print(f'Acuracia de {acuracia:.2f}')

Treino = 7500 e Teste = 2500
Acuracia de 63.20


In [8]:
from sklearn.dummy import DummyClassifier as dc

dummy_stratified = dc()
dummy_stratified.fit(treino_x, treino_y)
previsoes = dummy_stratified.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) *100
print(f'Acuracia do dummy: {acuracia:.2f}')

Acuracia do dummy: 52.44


In [9]:
dummy_mostfrequent = dc()
dummy_mostfrequent.fit(treino_x, treino_y)
previsoes = dummy_mostfrequent.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) *100
print(f'Acuracia do dummy: {acuracia:.2f}')

Acuracia do dummy: 50.68


In [10]:
from sklearn.preprocessing import StandardScaler as ss
from sklearn.svm import SVC as svc
from sklearn.metrics import accuracy_score as accuracy
from sklearn.model_selection import train_test_split as tt_split
import numpy as np

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = tt_split(x, y, test_size = .25, stratify = y)
print(f'Treino com {treino_x.shape[0]} elementos e teste com {teste_x.shape[0]} elementos')

scale = ss()
scale.fit(raw_treino_x)
treino_x = scale.transform(raw_treino_x)
teste_x = scale.transform(raw_teste_x)

modelo = svc(gamma = 'auto')
modelo.fit(treino_x, treino_y)

prev = modelo.predict(teste_x)
acuracia = accuracy(teste_y, prev)
print(f'A Acuracia foi de {acuracia*100:.2f}')

Treino com 7500 elementos e teste com 2500 elementos


C:\Users\guilh\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\guilh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\guilh\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


A Acuracia foi de 77.48


In [11]:
from sklearn.preprocessing import StandardScaler as ss
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.metrics import accuracy_score as accuracy
from sklearn.model_selection import train_test_split as tt_split
import numpy as np

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = tt_split(x, y, test_size = .25, stratify = y)
print(f'Treino com {treino_x.shape[0]} elementos e teste com {teste_x.shape[0]} elementos')

scale = ss()
scale.fit(raw_treino_x)
treino_x = scale.transform(raw_treino_x)
teste_x = scale.transform(raw_teste_x)

modelo = dtc()
modelo.fit(treino_x, treino_y)

prev = modelo.predict(teste_x)
acuracia = accuracy(teste_y, prev)
print(f'A Acuracia foi de {acuracia*100:.2f}')

Treino com 7500 elementos e teste com 2500 elementos
A Acuracia foi de 73.20


C:\Users\guilh\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\guilh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\guilh\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


In [12]:
from sklearn.tree import export_graphviz as export
import graphviz

dot_data = export(modelo, out_file = None)
grafico = graphviz.Source(dot_data)
grafico

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH